<a href="https://colab.research.google.com/github/GabrielMtzSoltero/wsEMBS/blob/main/workshopEMBS_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install mysql-connector

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 35.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mysql-connector: filename=mysql_connector-2.2.9-cp310-cp310-linux_x86_64.whl size=247952 sha256=3adc579f7b82de2557d2391abe99d19a387d7f3d95e26aa41597f91ea541b44b
  Stored in directory: /root/.cache/pip/wheels/76/48/9b/da67ff1a18fe8e9d428f9b1a177716d4a7d363d2bbe83bf6cf
Successfully built mysql-connector


In [19]:
import mysql.connector
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,  Activation

In [51]:
class DB:
    def __init__(self,_test=False):
        conexionDB = mysql.connector.connect(
        host="turtlechamber.com",
        user="embs",
        password="embs",
        database="breast_cancer"
        )
        self.conexionDB=conexionDB
        self.minRecord=1
        self.maxRecord=500
        if(_test):
          self.minRecord=501
          self.maxRecord=569
        np.random.seed(1)
        #self.getMinMaxRecords()
    #para obtener el primer y ultimo registro
    def getMinMaxRecords(self):
        cursor =self.conexionDB.cursor()
        query= "SELECT MIN(bc_id),MAX(bc_id) FROM breast_cancer"
        cursor.execute(query)
        resultado=cursor.fetchone()
        print(resultado)
        self.minRecord=resultado[0]
        self.maxRecord=resultado[1]

    def read_records_at(self, ids):
        cursor =self.conexionDB.cursor()
        valores=",".join(map(str, ids))
        query= "SELECT * FROM breast_cancer WHERE bc_id in ({})".format(valores)
        cursor.execute(query)
        result=cursor.fetchall()
        X=[]
        y=[]
        for row in result:
            X.append(row[2:])
            y.append([row[1]])
        X=np.array(X,dtype='float32')
        y=np.array(y,dtype='float32')
        return X, y

In [46]:
class DataGenerator(keras.utils.Sequence):
  def __init__(self, db, batch_size=32):
    self.db = db
    self.idx = np.arange(self.db.minRecord,self.db.maxRecord+1)
    self.batch_size = batch_size

  def __len__(self):
    return int(np.floor((self.db.maxRecord-self.db.minRecord) / self.batch_size))

  # Generate a batch of (X, y)
  def __getitem__(self, index):
    idxs = np.random.randint(self.db.minRecord,self.db.maxRecord+1,size=(self.batch_size))
    return self.db.read_records_at(idxs)

In [60]:
calis = DataGenerator(DB(), 2)
calis.__getitem__(2)

(array([[1.303e+01, 1.842e+01, 8.261e+01, 5.238e+02, 8.983e-02, 3.766e-02,
         2.562e-02, 2.923e-02, 1.467e-01, 5.863e-02, 1.839e-01, 2.342e+00,
         1.170e+00, 1.416e+01, 4.352e-03, 4.899e-03, 1.343e-02, 1.164e-02,
         2.671e-02, 1.777e-03, 1.330e+01, 2.281e+01, 8.446e+01, 5.459e+02,
         9.701e-02, 4.619e-02, 4.833e-02, 5.013e-02, 1.987e-01, 6.169e-02],
        [1.403e+01, 2.125e+01, 8.979e+01, 6.034e+02, 9.070e-02, 6.945e-02,
         1.462e-02, 1.896e-02, 1.517e-01, 5.835e-02, 2.589e-01, 1.503e+00,
         1.667e+00, 2.207e+01, 7.389e-03, 1.383e-02, 7.302e-03, 1.004e-02,
         1.263e-02, 2.925e-03, 1.533e+01, 3.028e+01, 9.827e+01, 7.155e+02,
         1.287e-01, 1.513e-01, 6.231e-02, 7.963e-02, 2.226e-01, 7.617e-02]],
       dtype=float32),
 array([[1.],
        [1.]], dtype=float32))

In [52]:
from keras.src.metrics import binary_accuracy
model = keras.models.Sequential()
model.add(Dense(64, activation='relu', input_dim=30))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer="adam",metrics=["accuracy",])


model.summary()
df = DataGenerator(DB(), 256)
dftest = DataGenerator(DB(_test=True))

model.fit(df,epochs=80)

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_71 (Dense)            (None, 64)                1984      
                                                                 
 dense_72 (Dense)            (None, 32)                2080      
                                                                 
 dense_73 (Dense)            (None, 1)                 33        
                                                                 
Total params: 4097 (16.00 KB)
Trainable params: 4097 (16.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/80
1/1 [==============================] - 1s 1s/step - loss: 99.0217 - accuracy: 0.6184 - binary_accuracy: 0.6184
Epoch 2/80
1/1 [==============================] - 0s 231ms/step - loss: 83.6239 - accuracy: 0.6089 - binary_accuracy: 0.6089
Epoch 3/80
1/1 [====================

In [56]:
y_pred=model.predict([[ 11.2, 29.37, 70.67, 386, 0.07449, 0.03558, 0, 0, 0.106, 0.05502, 0.3141, 3.896, 2.041, 22.81, 0.007594, 0.008878, 0, 0, 0.01989, 0.001773, 11.92, 38.3, 75.19, 439.6, 0.09267, 0.05494, 0, 0, 0.1566, 0.05905]])
y_pred

1/1 [==============================] - 0s 73ms/step


array([[0.97001666]], dtype=float32)

In [55]:
model.evaluate(dftest)

2/2 [==============================] - 1s 192ms/step - loss: 0.2797 - accuracy: 0.9200 - binary_accuracy: 0.9200


[0.27974313497543335, 0.9200000166893005, 0.9200000166893005]